In [1]:
import pandas as pd
import os
import cv2 as cv
import tensorflow as tf
from tensorflow.train import Example , Int64List , BytesList , FeatureList , Features 
import xml.etree.cElementTree as ET
from PIL import Image

# -----------------------------------
# Create jpg images from ppm images :

In [2]:
images_dir = '../data/FullIJCNN2013/'
images_info_dir = '../data/FullIJCNN2013/data_info.csv'
images_info_df = pd.read_csv(images_info_dir)
final_df = pd.DataFrame(columns=images_info_df.columns)

In [3]:
for current_dir , dirs ,files in os.walk(images_dir):
    for f in files:
        if f.endswith('.ppm'):
            image_name = f[:-4]
            image = cv.imread(images_dir + f)
            single_data_line = images_info_df.loc[images_info_df['path'] == f].copy()
            final_df = final_df.append(single_data_line)
            save_path = "../data/FullIJCNN2013/" + image_name + '.jpg'
            if not os.path.isfile(save_path):
                cv.imwrite(save_path,image)

In [4]:
final_df = final_df[~final_df.index.duplicated(keep='first')]
final_df.sort_index(inplace=True)

In [5]:
final_df.head(10)

,Unnamed: 0,path,left,top,right,bottom,id,object_name
0,0,00000.ppm,774,411,815,446,1,danger
1,1,00001.ppm,983,388,1024,432,2,mandatory
2,2,00001.ppm,386,494,442,552,2,mandatory
3,3,00001.ppm,973,335,1031,390,3,other
4,4,00002.ppm,892,476,1006,592,2,mandatory
5,5,00003.ppm,742,443,765,466,0,prohibitory
6,6,00003.ppm,742,466,764,489,0,prohibitory
7,7,00003.ppm,737,412,769,443,1,danger
8,8,00004.ppm,898,342,967,409,1,danger
9,9,00004.ppm,906,407,955,459,0,prohibitory


# -----------------------------------
# Creating annotations files :